### Recap from previous Lab

* We explored PyTorch (PT) tensor support and drew parallelisms w.r.t. NumPy `ndarray`
* We experimented with some basic Machine Learning (ML) building blocks in PT
* We built our first, very simple Artificial Neural Network (ANN), a MultiLayer Perceptron (MLP)

### Agenda for today

* Today we will be training a MLP using SGD and backpropagation
  * we will not be using the synthetic dataset we saw last lecture
  * we will be training our MLP on the MNIST dataset, which is a simple (in modern terms) dataset on which to train our ANNs. Due to this reputation, this problem is also called the *Hello World of Deep Learning*

### Intro to MNIST

MNIST is a dataset for **handwritten digit recognition**.

![](imgs/03/mnist.png)

* The dataset is composed of 60,000 grayscale images
  * by default, the dataset is already split into a training set of 50,000 images, while the remaining 10,000 images make up the test/validation set
* Each image is composed of 28x28 pixel
* Only one digit is present in each image
  * thus, we will be classifying digits from 0 to 9 (10 classes)
* The digit is centered within the image

#### Downloading the data

Since we're not covering the handling of data in this specific tutorial, I have prepared an external script which will download the data and "pack" it into DataLoaders.
You just need to know that DataLoaders and Datasets are two different entities; namely, DataLoaders are build on top of Datasets and handle the creation of the mini-batches that will later be fed into the MLP for the training and testing phase.

![](imgs/03/dataloader01.jpg)

The script returns the DataLoaders for both the training and the testing splits.

In [ ]:
import torch
from scripts import mnist
from matplotlib import pyplot as plt

minibatch_size_train = 256
minibatch_size_test = 512

trainloader, testloader = mnist.get_data(batch_size_train=minibatch_size_test, batch_size_test=minibatch_size_test)

Let us quickly view our data.

The dataset is an attribute of the dataloader. From here, we can access all the images of our dataset (at least for what concerns small datasets like MNIST; larger datasets are treated a little differently, we'll see more on that in the future labs).

The images are stored within the `data` attribute, while the labels lay in the `targets` attribute.

In [ ]:
plt.imshow(trainloader.dataset.data[15].numpy(), cmap="gray")

print("The label is", trainloader.dataset.targets[15].item())

By applying the `Tensor` skills we learned during the previous lab, we may also plot multiple images within the same plot.


In [ ]:
multi_img = trainloader.dataset.data[25:30]
multi_img.shape

We have a (5 x 28 x 28) `Tensor`, we can't plot it with `plt.imshow` because, for grayscale images, it needs a matrix as input.

**Q**: how can we solve this problem? Think about the methods we saw during the last lab

In [ ]:
# solve here
# shape 5*28 x 28
multi_img_reshaped = # your code here
plt.imshow(multi_img_reshaped.numpy(), cmap="gray")

#### Designing our MLP

Let us design our MLP.

We need to think about:

1. How wide is the output layer?
2. How wide is the input layer and how is it structured?
3. How many hidden layers and how wide need they be?
4. What activation functions will we be using?

You may try answering questions 1 and 2 by yourself as there's only one specific answer.
Q. 3 and 4, instead, are a choice operated by the designer (the data scientist) and there is no straight definite answer, as usually architectural features are treated as hyperparameters to be tuned as part of the training process.

For our MLP, then, we will be designing an MLP with
* 3 hidden layers
  * the first hidden layer has 16 neurons
  * the second hidden layer has 32 neurons
  * the third hidden layer has 24 neurons
* ReLU as activation function for the hidden layers ($ReLU(x) = \max(0, x)$)
* Cross Entropy loss (or, equivalently, Negative LogLikelihood loss)**\*\***

**\*\*** _Notice that, if you use NLLLoss, PyTorch wants as output of the model the **log** softmax (i.e., the **log probabilities**) of assignment to the classes, not the regular probabilities. This is for stability issues._

![](img/mlp_mnist.png)

In [ ]:
# write here the class for your MLP
class MLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.flat = torch.nn.Flatten() # X comes in as a n x 1 x 28 x 28 -> we need n 784-size vectors (or, a n x 784 matrix). flatten does this
        # your code here

    def forward(self, X): 
        # your code here
        out = X
        return out

#### The training loop

We now have to train the network using mini-batch Stochastic Gradient Descent (which we'll indicate as SGD).

The mini-batch part is already handled by the `DataLoader`.

We just need to call


```python
for X, y in dataloader:
    # do something
```

`X` and `y` represent our mini-batch (images and ground truth respectively).

* What do we need to do inside this loop?
  1. do the forward pass
  2. calculate the loss
  3. do the backward pass and update the parameters
* What objects do we need inside this loop?
  * the model (to do the forward pass)
  * the dataloader (to get `X, y`)
  * the loss function (to calculate the value of the loss and backpropagate)
  * the optimizer (to upgrade the value of the parameters given the gradients obtained via backprop)
* What do we need to do outside this loop?
  1. re-shuffle the mini-batches (NB: already taken care of by the `DataLoader`)
  2. repeat the loop for each epoch

Next, we are going to implement this loop (which we'll call `train_epoch`) in a minimalistic setting, i.e., with no excessive feature or embellishment.
  



In [ ]:
def train_epoch(model, dataloader, loss_fn, optimizer):
    for X, y in dataloader:
        # 1. reset the gradients previously accumulated by the optimizer
        #    this will avoid re-using gradients from previous loops
        optimizer.zero_grad() 
        # 2. get the predictions from the current state of the model
        #    this is the forward pass
        y_hat = model(X)
        # 3. calculate the loss on the current mini-batch
        loss = loss_fn(y_hat, y)
        # 4. execute the backward pass given the current loss
        loss.backward()
        # 5. update the value of the params
        optimizer.step()

#### Question (from last year's students): there's also a `model.zero_grad()` functionality. Is it the same as `optimizer.zero_grad()`?

Quick answer: usually yes. I have always seen `optimizer.zero_grad()` but some people prefer the former. See [here](https://discuss.pytorch.org/t/whats-the-difference-between-optimizer-zero-grad-vs-nn-module-zero-grad/59233) for discussion.

To work, we just need to wrap this routine inside another loop which will repeat `train_epoch` for each epoch.

In [ ]:
def train_model(model, dataloader, loss_fn, optimizer, num_epochs):
    # this is a useful switch that lets us pass to training phase to network evaluation
    # we will see in a future lab why it is necessary - for now it does nothing in particular
    model.train()
    for epoch in range(num_epochs):
        train_epoch(model, dataloader, loss_fn, optimizer)
        print(f"Epoch {epoch+1} completed.") # this is just so we have an idea on where we are during the training

We will use vanilla SGD with a learning rate of 0.001 and we'll run the training for 3 epochs.
Moreover, we will use the Negative Log-Likelihood (NLL) loss function.

In [ ]:
learn_rate = 0.1 # for SGD
num_epochs = 3

In [ ]:
model = MLP()
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learn_rate)

##### Training on GPU

Training on GPU requires three variables to reside on the same GPU:

1. the data
2. the labels
3. the parameters

they all can be achieved by calling the `.to(<device>)` method, which exists for both tensors and modules.
In the case for tensors, it does not operate in-place, but in the case of modules, the function moves all of the parameters without needing assignment.

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

print(f"We will use {device}")

In [ ]:
def train_epoch(model, dataloader, loss_fn, optimizer, device):
    for X, y in dataloader:
        ##### MOVE DATA AND LABELS TO THE DESIRED DEVICE #####
        X = X.to(device)
        y = y.to(device)
        
        # 1. reset the gradients previously accumulated by the optimizer
        #    this will avoid re-using gradients from previous loops
        optimizer.zero_grad() 
        # 2. get the predictions from the current state of the model
        #    this is the forward pass
        y_hat = model(X)
        # 3. calculate the loss on the current mini-batch
        loss = loss_fn(y_hat, y)
        # 4. execute the backward pass given the current loss
        loss.backward()
        # 5. update the value of the params
        optimizer.step()

In [ ]:
def train_model(model, dataloader, loss_fn, optimizer, num_epochs, device):
    # this is a useful switch that lets us pass to training phase to network evaluation
    # we will see in a future lab why it is necessary - for now it does nothing in particular
    model.train()
    model.to(device)
    for epoch in range(num_epochs):
        train_epoch(model, dataloader, loss_fn, optimizer)
        print(f"Epoch {epoch+1} completed.") # this is just so we have an idea on where we are during the training

In [ ]:
train_model(model, trainloader, loss_fn, optimizer, num_epochs, device)

The problem with this setting is that we have no idea how the network is faring, either on the training or the test set.

We need to devise some ways to assess the model.

Three ideas:

1. Accumulate the loss for each epoch
  * Alternatively, use the average loss per instance
2. Show the mean accuracy for each epoch
3. 1 and 2, but calculated on the test set

1 is easily implementable, just introduce a variable to accumulate the loss during the epoch and print it at the end of the training.

The average loss becomes a bit harder to implement.
Without reinventing the wheel, what is usually done is to make use of an auxiliary structure, which is called `AverageMeter`, to keep track of the running metrics and quickly summarize them.
Let' see how it's implemented:

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0 # val holds the current stat
        self.avg = 0 # avg holds the cumulative average
        self.sum = 0 # sum holds the cumulatie value
        self.count = 0 # count holds the number of instances seen

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

at the start of each epoch, we are going to re-instantiate one `AverageMeter` for keeping track of the loss.

In [ ]:
def train_epoch(model, dataloader, loss_fn, optimizer, loss_meter):
    for X, y in dataloader:
        ##### MOVE DATA AND LABELS TO THE DESIRED DEVICE #####
        X = X.to(device)
        y = y.to(device)
        # 1. reset the gradients previously accumulated by the optimizer
        #    this will avoid re-using gradients from previous loops
        optimizer.zero_grad() 
        # 2. get the predictions from the current state of the model
        #    this is the forward pass
        y_hat = model(X)
        # 3. calculate the loss on the current mini-batch
        loss = loss_fn(y_hat, y)
        # 4. execute the backward pass given the current loss
        loss.backward()
        # 5. update the value of the params
        optimizer.step()
        # 6. update the loss meter
        loss_meter.update(val=loss.item(), n=X.shape[0])

def train_model(model, dataloader, loss_fn, optimizer, num_epochs, device):
    model.train()
    model.to(device)
    for epoch in range(num_epochs):
        loss_meter = AverageMeter()
        train_epoch(model, dataloader, loss_fn, optimizer, loss_meter)
        print(f"Epoch {epoch+1} completed. Training loss: {loss_meter.avg}")

#### Assessing accuracy

Usually, $\text{loss} \in [0;+\infty)$, so it is hard to assess the effectiveness of the ANN based solely on such value.

Alongside a loss, we can assess the performance via a much more interpretable index as the **accuracy**

$\text{accuracy} = \frac{\text{\# correctly identified items}}{\text{\# all items}}$

To keep track of the accuracy, we can re-use the AverageMeter defined above, although we need to define a function for it---since PT does not implement it by default.

In [ ]:
def accuracy(y_hat, y):
    '''
    y_hat is the model output - a Tensor of shape (n x num_classes)
    y is the ground truth

    How can we implement this function?
    '''
    classes_prediction = y_hat.argmax(dim=1)
    match_ground_truth = classes_prediction == y # -> tensor of booleans
    correct_matches = match_ground_truth.sum()
    return (correct_matches / y_hat.shape[0]).item()
    

In [ ]:
def train_epoch(model, dataloader, loss_fn, optimizer, loss_meter, accuracy_meter):
    for X, y in dataloader:
        ##### MOVE DATA AND LABELS TO THE DESIRED DEVICE #####
        X = X.to(device)
        y = y.to(device)
        # 1. reset the gradients previously accumulated by the optimizer
        #    this will avoid re-using gradients from previous loops
        optimizer.zero_grad()
        # 2. get the predictions from the current state of the model
        #    this is the forward pass
        y_hat = model(X)
        # 3. calculate the loss on the current mini-batch
        loss = loss_fn(y_hat, y)
        # 4. execute the backward pass given the current loss
        loss.backward()
        # 5. update the value of the params
        optimizer.step()
        # 6. calculate the accuracy for this mini-batch
        acc = accuracy(y_hat, y)
        # 7. update the loss and accuracy AverageMeter
        loss_meter.update(val=loss.item(), n=X.shape[0])
        accuracy_meter.update(val=acc, n=X.shape[0])

def train_model(model, dataloader, loss_fn, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        loss_meter = AverageMeter()
        accuracy_meter = AverageMeter()
        train_epoch(model, dataloader, loss_fn, optimizer, loss_meter, accuracy_meter)
        # now with loss meter we can print both the cumulative value and the average value
        print(f"Epoch {epoch+1} completed. Loss - total: {loss_meter.sum} - average: {loss_meter.avg}; Accuracy: {accuracy_meter.avg}")
    # we also return the stats for the final epoch of training
    return loss_meter.sum, accuracy_meter.avg

*Additional observation*: we might wanna pass the `accuracy` as an additional `performance` parameter (analogous to `loss_fn`) since we might wanna evaluate the performance on a metric different than accuracy.

Now, we can train the network and examine its performance as we're running the training

In [ ]:
# reset the network and optimizer
model = MLP()
optimizer = torch.optim.SGD(model.parameters(), lr=learn_rate)

loss, acc = train_model(model, trainloader, loss_fn, optimizer, num_epochs)
print(f"Training completed - final accuracy {acc} and loss {loss}")

Recall that we wanted to assess also the network's performance on the test set. We need an additional testing routine for it.

For testing, we need only

1. the network
2. the test-set `DataLoader`
3. the loss function / the performance metrics

we don't need the optimizer as we're not updating the weights now


In [ ]:
def test_model(model, dataloader, device, loss_fn=None,):
    if loss_fn is not None:
        loss_meter = AverageMeter()
    acc_meter = AverageMeter()
    model.eval() # this is the contrary of model.train() we saw before - equivalent to model.train(False)
    model.to(device)
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            y_hat = model(X)
            loss = loss_fn(y_hat, y) if loss_fn is not None else None
            acc = accuracy(y_hat, y)
            if loss_fn is not None:
                loss_meter.update(loss.item(), X.shape[0])
            acc_meter.update(acc, X.shape[0])
    print(f"TESTING - loss {loss_meter.sum if loss_fn is not None else '--'} - accuracy {acc_meter.avg}")


In [ ]:
test_model(model, testloader)

Now that we have nice-enough-looking training and testing routines and that we have trained and tested our model we might want to save it so we can re-use it in the future.

The model can be easily saved using the `state_dict()` method

We can save it using PT built-in `torch.save`

In [ ]:
import os
folder_save = "models/mlp_mnist"
os.makedirs(folder_save, exist_ok=True)
filename = os.path.join(folder_save, "model.pt")
torch.save(model.state_dict(), filename)

**Disclaimer**: **NEVER** save the model with `torch.save(model)`. See reason [here](https://pytorch.org/tutorials/beginner/saving_loading_models.html) and [here](https://stackoverflow.com/questions/57407939/pytorch-saving-model-or-state-dict-gives-different-on-disk-space-occupation).

Let's suppose we wish to reload the state_dict. We just need one line of code: 

In [ ]:
model.load_state_dict(torch.load(filename))

#### Saving a model mid-training

Let's suppose we further need to train our model. Often times, storing only the weights may not be enough.

Till now, we have only seen examples in which the training hyperparameters are static. Modern techniques, though, require dynamic hyperparameters to ensure good performance or generalization. You may have heard of optimizers such as ADAM or techniques such as learning rate annealing. Thus, in order to restore the training, we need to save a **checkpoint** so that we're able to re-start training at the exact same conditions we were in when it first ended^.

For now, a checkpoint will be composed of the following objects:
1. The `state_dict` of the model
2. The state of the optimizer (which is also obtainable via a `state_dict` method---see code)
3. The epochs trained, so that we can select the epoch to restart with
4. (optionally) the iterations within the epoch, if we interrupted training mid-epoch (can be useful for very large models)
5. If you're using a learning rate schedule (we'll see in a few labs what this is), also its state dict.


^ *Note that, due to the intrinsic stochasticity of SGD, we may not actually be able to reproduce the exact same situation as before.*

In [ ]:
checkpoint_dict = {
    "parameters": model.state_dict(),
    "optimizer": optimizer.state_dict(),
    "epoch": ...,
    "iteration": ...,
    "lr_scheduler": ...
}

In [ ]:
# save
filename = os.path.join(folder_save, "checkpoint.pt")
torch.save(checkpoint_dict, filename)

In [ ]:
# restore checkpoint
checkpoint = torch.load(filename)
model.load_state_dict(checkpoint["parameters"])
optimizer.load_state_dict(checkpoint["optimizer"])

In a setting where you wish to restart training, you may modify the `train_model` function by adding the argument `epoch_restart` and looping through epochs like this
```python
for epoch in range(epoch_restart, num_epochs):
    # train loop
```

if training broken during a random iteration:

```python
for X, y in trainloader:
    # what do I write here?
```

#### Training a model on custom loss functions

We might want to fit our MLP using custom losses. Despite being the *de facto* choice for multiclass ML problems, Cross Entropy is not the single loss we can use.

A very simple loss we may consider is the Quadratic Loss (QL) [1](https://www.semanticscholar.org/paper/Exploring-the-Role-of-Loss-Functions-in-Multiclass-Demirkaya-Chen/5c80d4ce2ce5e4f0b395969618cd76d35210e72c):

$QL(\hat{y}, y) = \frac{1}{2}\text{E}[\vert\vert y^{(\text{one-hot})} - \hat{y} \vert\vert^2] 
 = \frac{1}{2n}\sum_{i=1}^{n} (\vert\vert y^{(\text{one-hot})}_i - \hat{y}_i \vert\vert)^2
 = \frac{1}{2n}\sum_{i=1}^{n}\sum_{j=1}^{d}(y^{(\text{one-hot})}_{i,j} - \hat{y}_{i,j})^2$

Where:
* the one-hot encoding for $y=c$, where $c$ is a possible class out of $C$ classes, is the column vector whose elements are all $0$ beside the $c$-th entry, which is $1$;
* with $\hat{y}$ we mean the output of the model, i.e. a vector representing a probability distribution that a data point be assigned to each class.

For instance, given a data point whose ANN output is $\hat{y}=[0.3, 0.2, 0.4, 0.1]^\top$ and the ground truth is $y=3$, the QL for this point is

$0.5\cdot \vert\vert[0, 0, 1, 0]^\top - [0.3, 0.2, 0.4, 0.1]^\top\vert\vert^2 = 0.5\cdot \vert\vert[-0.3, -0.2, 0.6, -0.1]^\top\vert\vert^2
= 0.5\cdot (0.09 + 0.04 + 0.36 + 0.01) = 0.5\cdot 0.5 = 0.25$

Let us implement this loss function in PT

In [ ]:
y = torch.Tensor([0,1,2,3,0,0]).long()
y_onehot = torch.nn.functional.one_hot(y, num_classes=5)
print(y_onehot)
print("Shape", y_onehot.shape)

In [ ]:
def quadratic_loss(y_hat, y):
    '''
    y_hat is a matrix of dimension (n x C),
    where C is the number of classes, and n is the number of datapoints
    y is a vector of classes
    '''
    # convert y to onehot
    y_onehot = torch.nn.functional.one_hot(y.long())
    y_hat = y_hat ##HERE WE MUST DO SOMETHING WITH OUR DATA DEPENDING ON THE FINAL ACTIVATION FUNCTION OF OUR MODEL
    norm = (y_onehot - y_hat.exp()).norm(dim=1)**2
    norm_sum = norm.sum()
    return norm_sum / (2*y_hat.shape[1])

Let's be more PyTorch-ian:

In [ ]:
class QLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, y_hat, y):
        y_onehot = torch.nn.functional.one_hot(y.long())
        norm_square = (y_onehot - y_hat.exp()).norm(dim=1)**2
        norm_sum = norm_square.sum()
    


In [ ]:
learn_rate = 0.1
num_epochs = 3

model = MLP()
loss_fn = quadratic_loss
optimizer = torch.optim.SGD(model.parameters(), lr=learn_rate)

In [ ]:
train_model(model, trainloader, loss_fn, optimizer, num_epochs)

**Q**: how were we able to train the network on our custom loss even if we never told PT how to compute the backward pass on it (i.e., how to compute its gradient)?

### Custom activation functions

Let us suppose we wish to add a custom activation function to our network, for example, the sine function.

We'll quickly discover that we can't just add a functional representation of it to our `Sequential`. Let's try it

In [ ]:
class SineNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.structure = torch.nn.Sequential(
            torch.nn.Linear(2, 2),
            torch.sin # this won't work!
        )
    
    def forward(self, X):
        return self.structure(X)

nn = SineNN()

In fact, PyTorch is complaining that the sin module is not inheriting from `torch.nn.Module`, which it needs to do. So, we only need to wrap `torch.sin` in a class like so.

In [ ]:
class Sin(torch.nn.Module):
    # it needs to have at least a constructor and a forward method, just like an MLP
    def __init__(self): 
        super().__init__()
        # no additional stuff to do here as Sin has no additional parameters to set
    
    def forward(self, X):
        return torch.sin(X)

In [ ]:
# it will not produce an exeption now
class SineNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.structure = torch.nn.Sequential(
            torch.nn.Linear(2, 2),
            Sin()
        )
    
    def forward(self, X):
        return self.structure(X)

nn = SineNN()

**Homework** _(not compulsory)_

- Reconstruct in PyTorch the first experiment in [Learning representations by back-propagating errors](https://www.nature.com/articles/323533a0) with learning rule in eq.8 (gradient descent without momentum---you can re-use `torch.optim.SGD` with the appropriate keyword)
  - Try to be as close as possible to the original protocol, except for what regards the learning rule
  - Read the paper, if you did not do it yet (don’t worry if you don’t understand the other experiments in detail)
  - Create the data, the model and everything is needed (do not use dataloaders if you don’t know how yet how they work)
  - Train the model
  - Inspect the weights you obtained and check if they provide a solution to the problem
  - Compare the solution to the solution reported in the paper
Tip: don't expect to get a fully working implementation!


Additional tips for implementing ANNs in PyTorch (will not be part of the exam): [PyTorch common mistakes, by Aladdin Person](https://youtu.be/O2wJ3tkc-TU)

## References
[1](https://www.semanticscholar.org/paper/Exploring-the-Role-of-Loss-Functions-in-Multiclass-Demirkaya-Chen/5c80d4ce2ce5e4f0b395969618cd76d35210e72c) A. Demirkaya, J. Chen and S. Oymak, "Exploring the Role of Loss Functions in Multiclass Classification," 2020 54th Annual Conference on Information Sciences and Systems (CISS), Princeton, NJ, USA, 2020, pp. 1-5, doi: 10.1109/CISS48834.2020.1570627167.